In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold
from utils.eval import get_results
import json
from monai.networks.nets import DenseNet121
import torch
from monai.data import DataLoader, ImageDataset
from monai.transforms import RandRotate90, Resize, EnsureChannelFirst, Compose, ScaleIntensity,RandAxisFlip
import os
from tqdm import tqdm

C:\Users\sugar\AppData\Local\Temp\ipykernel_12468\2607817645.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Load Random forest model 

In [3]:
# Get features
features_2d = pd.read_csv('./Data/2D_t2/all.csv')
data = features_2d.drop(columns=['Center'])
correlation = data.iloc[:,2:].corr(method='spearman')
selected_features = correlation[(correlation.abs() <= 0.6) | (correlation.abs() == 1)]
for i in range(200):
    for index in range(selected_features.shape[0]):
        row = selected_features.iloc[index]
        to_drop = row[row.isna()].index
        selected_features_c_droped = selected_features.drop(labels=to_drop,axis=0)
        selected_features_r_droped = selected_features_c_droped.drop(labels=to_drop,axis=1)
        if selected_features_r_droped.shape[0]<selected_features.shape[0]:
            break
    selected_features = selected_features_r_droped
selected_data = data[selected_features.index.to_list()+['Label','Name']]

Organize label

In [4]:
selected_data['Label'] = selected_data['Label'].apply(lambda x: x-1)

C:\Users\sugar\AppData\Local\Temp\ipykernel_12468\1549703430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['Label'] = selected_data['Label'].apply(lambda x: x-1)


Read radiomics data

In [ ]:
with open("Fold_data.json", "r") as file:
    fold_data = json.load(file)
# Radiomics features for training and testing
train_list = fold_data['Fold1']['train']
test_list = fold_data['Fold1']['test']
train_data = selected_data[selected_data["Name"].isin(train_list)]
test_data = selected_data[selected_data["Name"].isin(test_list)]

# test label for 
test_label_and_path = test_data[['Name','Label']]
test_label_and_path['Name'] = test_label_and_path['Name'].apply(lambda x: x + '.nii.gz')

Fit random forest

Load DenseNet121

In [ ]:
test_label_and_path = test_data[['Name','Label']]
test_label_and_path['Name'] = test_label_and_path['Name'].apply(lambda x: x + '.nii.gz')

C:\Users\sugar\AppData\Local\Temp\ipykernel_18656\1218906158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label_and_path['Name'] = test_label_and_path['Name'].apply(lambda x: x + '.nii.gz')


In [ ]:


model = DenseNet121(
        spatial_dims=3,  # 3D input
        in_channels=1,   # Typically for grayscale (e.g., MRI/CT scans), change to 3 for RGB
        out_channels=2   # Adjust for binary or multi-class segmentation/classification
    )
model.load_state_dict(torch.load('./weights/fold1_best_auc_model.pth', map_location='cpu', weights_only=True))
model.to('cuda')
data_dir  = r"D:\DATASET\IPMN_t2_cyst\ROI"
test_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96))])

test_ds = ImageDataset(image_files=[os.path.join(data_dir,name) for name in test_label_and_path['Name'].to_list()], labels=test_label_and_path['Label'].to_list(), transform=test_transforms)
test_dataloader = DataLoader(test_ds, batch_size=1, shuffle=False,num_workers=1)

In [14]:

with torch.no_grad(): 
    progress_bar = tqdm(test_dataloader, desc="Testing")
    for X, y in progress_bar:
        X = X.to('cuda')
        pred = model(X)
        print(torch.nn.functional.softmax(pred, dim=-1).cpu().numpy())

NameError: name 'test_dataloader' is not defined